In [ ]:
import geopy
import geopy.distance
import time
import googlemaps 
import pandas as pd
import datetime

# API Key input
API_KEY = 'Your API Key here'
gmaps = googlemaps.Client(key=API_KEY)

## Read CSV
df = pd.read_csv('api_radius.csv', sep = ';', encoding='cp1252')

## Converting to numeric
cols = ['raio']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

## Empty lists and bearings
coordinates_new = [[] for i in range(len(df['unidade']))]
points = [[] for i in range(len(df['unidade']))]
duration = [[] for i in range(len(df['unidade']))]
lat = []
lon = []
bearings = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]

## Spliting Latitude and Longitude into two separate columns
for row in df['lat_lon']:
    lat.append(row.split(',')[0])
    lon.append(row.split(', ')[1])
    
df['lat'] = lat
df['lon'] = lon

## Getting lat/lon coordinates for each school radius 
for i in range(df.shape[0]):
    start = geopy.Point(df['lat'][i], df['lon'][i])
    d = geopy.distance.distance(meters=df['raio'][i])
    
    for j in bearings:
        final = d.destination(point=start, bearing=j)
        points[i].append(final)
        
## Calculating time in seconds
for i in range(len(points)):
    for j in range(len(points[i])):
        coordinates_new[i].append(str(points[i][j][0]) + ',' + str(points[i][j][1]))

#for i in range(len(df)):
for i in range(df.shape[0]):
    for j in range(len(coordinates_new[i])):
        try:
            duration[i].append(gmaps.distance_matrix(df['lat_lon'][i],
                                  coordinates_new[i][j], 
                                  mode='driving')["rows"][0]["elements"][0]["duration"]["value"])
        except KeyError:
            print('Missing Data')
            pass
    
def average(lst):
    return sum(lst) / len(lst)

df['time_seconds_drive'] = [average(duration[i]) for i in range(len(duration))]        
